In [1]:
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
import json
pd.set_option("display.max.columns", None)

In [23]:
# Obtain the list of id's for the webpages to scrape
iedbfile = pd.read_csv('iedb_data.csv', sep=',', error_bad_lines=False, dtype='unicode')
new_header = iedbfile.iloc[0] 
iedbfile = iedbfile[1:] 
iedbfile.columns = new_header
listofIds = iedbfile['T Cell ID'].tolist()
#iedbfile[iedbfile['T Cell ID'] == '3968896']
iedbfile.head(5)

/home/mrcai/.local/lib/python3.6/site-packages/pandas/core/indexes/base.py:2646: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return self._engine.get_loc(key)


TypeError: Cannot convert bool to numpy.ndarray

In [26]:
# Extracting TCRs
# https://www.iedb.org/assay/"T Cell ID" <- webpage for each of these
baseURL = 'https://www.iedb.org/assay/'
proteinURLs = {}

count = 0
listofIds = ['3968896']
for tId in listofIds:
    try:
        r = requests.get(baseURL + tId)
        soup = BeautifulSoup(r.content,'lxml')
        pattern = 'var resultData.*};'
        a = re.search(pattern, str(soup))
        extractedJson = re.sub('^var resultData = ', '', a.group())
        epitope = json.loads(extractedJson[:-1])['ant_mol1_seq']['value']
        proteinURLs[epitope] = json.loads(extractedJson[:-1])['tcr_mol1_name']['identifier'].replace('GenPept:', '')
    except:
        pass

In [30]:
# Find proteins on NCBI
baseURL = 'https://www.ncbi.nlm.nih.gov/protein/'
for identifier in proteinURLs:
    r = requests.get(baseURL + proteinURLs[identifier])
    soup = BeautifulSoup(r.content,'lxml')
soup

<?xml version="1.0" encoding="utf-8"?><!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
<head xmlns:xi="http://www.w3.org/2001/XInclude"><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<!-- meta -->
<meta content="index,nofollow,noarchive" name="robots"/>
<meta content="entrez" name="ncbi_app"/><meta content="protein" name="ncbi_db"/><meta content="genpept" name="ncbi_report"/><meta content="html" name="ncbi_format"/><meta content="20" name="ncbi_pagesize"/><meta content="default" name="ncbi_sortorder"/><meta content="1" name="ncbi_pageno"/><meta content="1" name="ncbi_resultcount"/><meta content="retrieve" name="ncbi_op"/><meta content="genpept" name="ncbi_pdid"/><meta content="CE889A90ED71E821_1917SID" name="ncbi_sessionid"/><meta content="1397983485" name="ncbi_uidlist"/><meta content="all" name="ncbi_filter"/><meta content=